###Reading: https://magazine.sebastianraschka.com/p/coding-the-kv-cache-in-llms
###Reading https://medium.com/data-scientists-diary/advanced-guide-to-using-nn-modulelist-in-pytorch-da4d49c109fc

In [1]:
import time
import tiktoken
import torch
import torch.nn as nn

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1),
            persistent=False
        )
        ###NEW#####################################################################################
        self.register_buffer("k_cache", None, persistent = False)
        self.register_buffer("v_cache", None, persistent = False)
        self.current_pos = 0
        ###########################################################################################

    def reset_cache(self):
        self.cache_k, self.cache_v = None, None

    def forward(self, x, uses_cache=False):
        b, num_tokens, d_in = x.shape

        new_keys = self.W_key(x)  # Shape: (b, num_tokens, d_out)
        new_values = self.W_value(x)
        queries = self.W_query(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        new_keys = new_keys.view(b, num_tokens, self.num_heads, self.head_dim)
        new_values = new_values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        ########################################################################################
        #NEW

        if uses_cache:
            self.k_cache = torch.cat((self.k_cache, new_keys), dim=1) if self.k_cache is not None else new_keys
            self.v_cache = torch.cat((self.v_cache, new_values), dim=1) if self.v_cache is not None else new_values
            keys = self.k_cache
            values = self.v_cache
        else:
            keys = new_keys
            values = new_values
        ###########################################################################################

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        ########################################################################################
        #NEW

        Q_num_tokens = queries.shape[-2]
        K_num_tokens = keys.shape[-2]
        if uses_cache:
            mask_bool = self.mask.bool()[self.current_pos : self.current_pos + Q_num_tokens,:K_num_tokens]
            self.current_pos += Q_num_tokens
        else:
            mask_bool = self.mask.bool()[:Q_num_tokens, :K_num_tokens]
        ########################################################################################

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec

In [3]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [4]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [5]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [6]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x, uses_cache=False):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        ####################################################
        # NEW
        x = self.att(x, uses_cache)   # Shape [batch_size, num_tokens, emb_size]
        ####################################################
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed-forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

In [13]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        ####################################################
        # NEW
        self.trf_blocks = nn.ModuleList( [TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        self.current_pos = 0
        ####################################################

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx, uses_cache=False):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)

        pos_embeds = None
        ####################################################
        # NEW
        if uses_cache:
            pos_embeds = self.pos_emb(torch.arange(self.current_pos, self.current_pos + seq_len, device=in_idx.device ,dtype=torch.long))
            self.current_pos += seq_len
        else:
            pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device ,dtype=torch.long))
        ####################################################

        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        ####################################################
        # NEW
        for block in self.trf_blocks:
            x = block(x, uses_cache = uses_cache)
        ####################################################

        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

    ####################################################
    # NEW
    def reset_kv_cache(self):
        for blk in self.trf_blocks:
            blk.att.reset_cache()
        self.current_pos = 0
    ####################################################

In [8]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):

        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]

        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)

        # Focus only on the last time step
        # (batch, n_token, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]

        # Get the idx of the vocab entry with the highest logits value
        idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

In [14]:
def generate_text_simple_cached(model, idx, max_new_tokens, context_size = None, uses_cache = False):
    model.eval()
    # idx is (B, T) array of indices in the current context
    context_len = context_size or model.pos_emb.num_embeddings

    if uses_cache:
        #PREFILL
        model.reset_kv_cache()
        idx_cond = idx[:, -context_len:]

        for _ in range(max_new_tokens):

            # Get the predictions
            with torch.no_grad():
                logits = model(idx_cond, uses_cache = True)

            # (batch, n_token, vocab_size) becomes (batch, vocab_size)
            logits = logits[:, -1, :]

            # Get the idx of the vocab entry with the highest logits value
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch, 1)
            idx_cond = idx_next
            idx = torch.cat((idx, idx_next), dim=1)
    else:
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -context_len:]
            with torch.no_grad():
                logits = model(idx_cond, uses_cache = uses_cache)
            logits = logits[:, -1, :]
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch, 1)
            idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

In [15]:
def main(uses_cache = False):
    GPT_CONFIG_124M = {
        "vocab_size": 50257,     # Vocabulary size
        "context_length": 1024,  # Context length
        "emb_dim": 768,          # Embedding dimension
        "n_heads": 12,           # Number of attention heads
        "n_layers": 12,          # Number of layers
        "drop_rate": 0.1,        # Dropout rate
        "qkv_bias": False        # Query-Key-Value bias
    }

    torch.manual_seed(123)
    model = GPTModel(GPT_CONFIG_124M)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()  # disable dropout

    start_context = "Hello, I am"

    tokenizer = tiktoken.get_encoding("gpt2")
    encoded = tokenizer.encode(start_context)
    encoded_tensor = torch.tensor(encoded, device=device).unsqueeze(0)

    print(f"\n{50*'='}\n{22*' '}IN\n{50*'='}")
    print("\nInput text:", start_context)
    print("Encoded input text:", encoded)
    print("encoded_tensor.shape:", encoded_tensor.shape)

    if torch.cuda.is_available():
        torch.cuda.synchronize()
    start = time.time()

    # token_ids = generate_text_simple(
    #     model=model,
    #     idx=encoded_tensor,
    #     max_new_tokens=200,
    #     context_size=GPT_CONFIG_124M["context_length"]
    # )

    ####################################################
    # NEW
    token_ids = generate_text_simple_cached(
        model=model,
        idx=encoded_tensor,
        max_new_tokens=200,
        uses_cache=uses_cache
    )
    ####################################################

    if torch.cuda.is_available():
        torch.cuda.synchronize()
    total_time = time.time() - start

    decoded_text = tokenizer.decode(token_ids.squeeze(0).tolist())

    print(f"\n\n{50*'='}\n{22*' '}OUT\n{50*'='}")
    print("\nOutput:", token_ids)
    print("Output length:", len(token_ids[0]))
    print("Output text:", decoded_text)

    print(f"\nTime: {total_time:.2f} sec")
    print(f"{int(len(token_ids[0])/total_time)} tokens/sec")
    if torch.cuda.is_available():
        max_mem_bytes = torch.cuda.max_memory_allocated()
        max_mem_gb = max_mem_bytes / (1024 ** 3)
        print(f"Max memory allocated: {max_mem_gb:.2f} GB")

In [16]:
main()


                      IN

Input text: Hello, I am
Encoded input text: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


                      OUT

Output: tensor([[15496,    11,   314,   716, 27018, 24086, 47843, 30961, 42348,  7267,
         49706, 43231, 47062, 34657, 18631, 49188, 43312, 45933, 23154, 15983,
         10345, 16369, 46214, 22954, 34674, 21100,  4743, 14056, 42526,  6459,
         12799,  5734, 49274,   136, 49294, 42900, 21193, 20463,  1018,  7864,
         13895, 27167, 12810, 25727, 14388,   985, 15797, 24440, 18557, 48625,
         10579,  4007, 11895, 45365, 19051,  1355, 47705,  5120, 32858, 49293,
          5141, 22900, 36570, 22215, 16369, 25803,  9254, 33694, 23188, 21624,
         12696,  1697, 12315, 23338,  1361, 49487, 27970, 21641, 28170, 36226,
          8980, 34715, 15683, 21370,   829, 41165, 19250, 40921, 47972, 29169,
         17681, 13937,   719,  7781, 46519, 39685, 35637, 38254, 37355, 48054,
          6960, 32389, 49945, 48307, 43

In [17]:
main(uses_cache=True)


                      IN

Input text: Hello, I am
Encoded input text: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


                      OUT

Output: tensor([[15496,    11,   314,   716, 27018, 24086, 47843, 30961, 42348,  7267,
         49706, 43231, 47062, 34657, 18631, 49188, 43312, 45933, 23154, 15983,
         10345, 16369, 46214, 22954, 34674, 21100,  4743, 14056, 42526,  6459,
         12799,  5734, 49274,   136, 49294, 42900, 21193, 20463,  1018,  7864,
         13895, 27167, 12810, 25727, 14388,   985, 15797, 24440, 18557, 48625,
         10579,  4007, 11895, 45365, 19051,  1355, 47705,  5120, 32858, 49293,
          5141, 22900, 36570, 22215, 16369, 25803,  9254, 33694, 23188, 21624,
         12696,  1697, 12315, 23338,  1361, 49487, 27970, 21641, 28170, 36226,
          8980, 34715, 15683, 21370,   829, 41165, 19250, 40921, 47972, 29169,
         17681, 13937,   719,  7781, 46519, 39685, 35637, 38254, 37355, 48054,
          6960, 32389, 49945, 48307, 43